In [1]:
#!pip install selenium
#!pip install chromedriver-binary-auto
#!pip install webdriver-manager

In [2]:
import time

from selenium import webdriver
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import os
import requests

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
#options = Options()
#options.add_argument('--headless')    # ヘッドレスモードに
#driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(driver,10)

URL = "https://db.netkeiba.com/?pid=race_search_detail"
driver.get(URL)
time.sleep(1)
wait.until(EC.presence_of_all_elements_located)

#ダートの選択
dart_element = driver.find_element(By.CSS_SELECTOR,"#check_track_2")
dart_element.click()


# 月ごとに検索
year = 2018
month = 1
endyear = 2018
endmonth = 12

# 期間を選択
#start_year_element = driver.find_element_by_name('start_year')
start_year_element =driver.find_element(By.NAME,"start_year")
start_year_select = Select(start_year_element)
start_year_select.select_by_value(str(year))
#start_mon_element = driver.find_element_by_name('start_mon')
start_mon_element = driver.find_element(By.NAME,"start_mon")
start_mon_select = Select(start_mon_element)
start_mon_select.select_by_value(str(month))
#end_year_element = driver.find_element_by_name('end_year')
end_year_element = driver.find_element(By.NAME,"end_year")
end_year_select = Select(end_year_element)
end_year_select.select_by_value(str(endyear))
#end_mon_element = driver.find_element_by_name('end_mon')
end_mon_element = driver.find_element(By.NAME,"end_mon")
end_mon_select = Select(end_mon_element)
end_mon_select.select_by_value(str(endmonth))

# 中央競馬場をチェック
#for i in range(1,11):
    #terms = driver.find_element_by_id("check_Jyo_"+ str(i).zfill(2))
#    terms = driver.find_element(By.ID,"check_Jyo_"+ str(i).zfill(2))
#    terms.click()

# 東京競馬場をチェック
#tokyo_element = driver.find_element(By.CSS_SELECTOR,"#check_Jyo_05")
tokyo_element = driver.find_element(By.ID,"check_Jyo_"+ str(5).zfill(2))
tokyo_element.click()
        
# 表示件数を選択(20,50,100の中から最大の100へ)
#list_element = driver.find_element_by_name('list')
list_element = driver.find_element(By.NAME,"list")
list_select = Select(list_element)
list_select.select_by_value("100")

# フォームを送信
#frm = driver.find_element_by_css_selector("#db_search_detail_form > form")
frm = driver.find_element(By.CSS_SELECTOR,"#db_search_detail_form > form")
frm.submit()
time.sleep(5)
wait.until(EC.presence_of_all_elements_located)

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.79M/6.79M [00:00<00:00, 59.8MB/s]
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\919600567.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


<function selenium.webdriver.support.expected_conditions.presence_of_all_elements_located.<locals>._predicate(driver)>

In [4]:
with open(str(year)+"-"+str(month)+".txt", mode='w') as f:
    while True:
        time.sleep(5)
        wait.until(EC.presence_of_all_elements_located)
        #all_rows = driver.find_element_by_class_name('race_table_01').find_elements_by_tag_name("tr")
        all_rows = driver.find_element(By.CLASS_NAME,"race_table_01").find_elements(By.TAG_NAME,"tr")
        for row in range(1, len(all_rows)):
            #race_href=all_rows[row].find_elements_by_tag_name("td")[4].find_element_by_tag_name("a").get_attribute("href")
            race_href=all_rows[row].find_elements(By.TAG_NAME,"td")[4].find_element(By.TAG_NAME,"a").get_attribute("href")
            f.write(race_href+"\n")
        try:
            #target = driver.find_elements_by_link_text("次")[0]
            target = driver.find_elements(By.LINK_TEXT,"次")[0]
            driver.execute_script("arguments[0].click();", target) #javascriptでクリック処理
        except IndexError:
            break

In [5]:
import codecs
save_dir = "html"+"/"+str(year)+"/"+str(month)
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
        
with codecs.open(str(year)+"-"+str(month)+".txt", "r",encoding="utf-8-sig") as f:
    urls = f.read().splitlines()
    for url in urls:
        list = url.split("/")
        race_id = list[-2]
        save_file_path = save_dir+"/"+race_id+'.html'
        response = requests.get(url)
        response.encoding = response.apparent_encoding
        html = response.text
        time.sleep(5)
        with codecs.open(save_file_path, 'w',encoding="utf-8-sig") as file:
            file.write(html)

In [6]:
race_data_columns=[
    'race_id',
    'race_round',
    'race_title',
    'race_course',
    'weather',
    'ground_status',
    'time',
    'date',
    'where_racecourse',
    'total_horse_number',
    'frame_number_first',
    'horse_number_first',
    'frame_number_second',
    'horse_number_second',
    'frame_number_third',
    'horse_number_third',
    'tansyo',
    'hukusyo_first',
    'hukusyo_second',
    'hukusyo_third',
    'wakuren',
    'umaren',
    'wide_1_2',
    'wide_1_3',
    'wide_2_3',
    'umatan',
    'renhuku3',
    'rentan3'
    ]

horse_data_columns=[
    'race_id',
    'rank',
    'frame_number',
    'horse_number',
    'horse_id',
    'sex_and_age',
    'burden_weight',
    'rider_id',
    'goal_time',
    'goal_time_dif',
    'time_value',
    'half_way_rank',
    'last_time',
    'odds',
    'popular',
    'horse_weight',
    'tame_time',
    'tamer_id',
    'owner_id'
]


def get_rade_and_horse_data_by_html(race_id, html):
    race_list = [race_id]
    horse_list_list = []
    soup = BeautifulSoup(html, 'html.parser')

    # race基本情報
    data_intro = soup.find("div", class_="data_intro")
    race_list.append(data_intro.find("dt").get_text().strip("\n")) # race_round
    race_list.append(data_intro.find("h1").get_text().strip("\n")) # race_title
    race_details1 = data_intro.find("p").get_text().strip("\n").split("\xa0/\xa0")
    race_list.append(race_details1[0]) # race_course
    race_list.append(race_details1[1]) # weather
    race_list.append(race_details1[2]) # ground_status
    race_list.append(race_details1[3]) # time
    race_details2 = data_intro.find("p", class_="smalltxt").get_text().strip("\n").split(" ")
    race_list.append(race_details2[0]) # date
    race_list.append(race_details2[1]) # where_racecourse


    result_rows = soup.find("table", class_="race_table_01 nk_tb_common").findAll('tr') # レース結果
    # 上位3着の情報
    race_list.append(len(result_rows)-1) # total_horse_number
    for i in range(1,4):
        row = result_rows[i].findAll('td')
        race_list.append(row[1].get_text()) # frame_number_first or second or third
        race_list.append(row[2].get_text()) # horse_number_first or second or third


    # 払い戻し(単勝・複勝・三連複・3連単)
    pay_back_tables = soup.findAll("table", class_="pay_table_01")

    pay_back1 = pay_back_tables[0].findAll('tr') # 払い戻し1(単勝・複勝)
    race_list.append(pay_back1[0].find("td", class_="txt_r").get_text()) #tansyo
    hukuren = pay_back1[1].find("td", class_="txt_r")
    tmp = []
    for string in hukuren.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # 枠連
    try:
        race_list.append(pay_back1[2].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")

    # 馬連
    try:
        race_list.append(pay_back1[3].find("td", class_="txt_r").get_text())
    except IndexError:
        race_list.append("0")



    pay_back2 = pay_back_tables[1].findAll('tr') # 払い戻し2(三連複・3連単)

    # wide 1&2
    wide = pay_back2[0].find("td", class_="txt_r")
    tmp = []
    for string in wide.strings:
        tmp.append(string)
    for i in range(3):
        try:
            race_list.append(tmp[i]) # hukuren_first or second or third
        except IndexError:
            race_list.append("0")

    # umatan
    race_list.append(pay_back2[1].find("td", class_="txt_r").get_text()) #umatan

    race_list.append(pay_back2[2].find("td", class_="txt_r").get_text()) #renhuku3
    try:
        race_list.append(pay_back2[3].find("td", class_="txt_r").get_text()) #rentan3
    except IndexError:
        race_list.append("0")

    # horse data
    for rank in range(1, len(result_rows)):
        horse_list = [race_id]
        result_row = result_rows[rank].findAll("td")
        # rank
        horse_list.append(result_row[0].get_text())
        # frame_number
        horse_list.append(result_row[1].get_text())
        # horse_number
        horse_list.append(result_row[2].get_text())
        # horse_id
        horse_list.append(result_row[3].find('a').get('href').split("/")[-2])
        # sex_and_age
        horse_list.append(result_row[4].get_text())
        # burden_weight
        horse_list.append(result_row[5].get_text())
        # rider_id
        horse_list.append(result_row[6].find('a').get('href').split("/")[-2])
        # goal_time
        horse_list.append(result_row[7].get_text())
        # goal_time_dif
        horse_list.append(result_row[8].get_text())
        # time_value(premium)
        horse_list.append(result_row[9].get_text())
        # half_way_rank
        horse_list.append(result_row[10].get_text())
        # last_time(上り)
        horse_list.append(result_row[11].get_text())
        # odds
        horse_list.append(result_row[12].get_text())
        # popular
        horse_list.append(result_row[13].get_text())
        # horse_weight
        horse_list.append(result_row[14].get_text())
        # tame_time(premium)
        horse_list.append(result_row[15].get_text())
        # 16:コメント、17:備考
        # tamer_id
        horse_list.append(result_row[18].find('a').get('href').split("/")[-2])
        # owner_id
        horse_list.append(result_row[19].find('a').get('href').split("/")[-2])

        horse_list_list.append(horse_list)

    return race_list, horse_list_list

CSV_DIR = "csv"
if not os.path.isdir(CSV_DIR):
    os.makedirs(CSV_DIR)
save_race_csv = CSV_DIR+"/race-"+str(year)+"-"+str(month)+".csv"
horse_race_csv = CSV_DIR+"/horse-"+str(year)+"-"+str(month)+".csv"

# race_data_columns, horse_data_columnsは長くなるので省略
race_df = pd.DataFrame(columns=race_data_columns )
horse_df = pd.DataFrame(columns=horse_data_columns )

html_dir = "html"+"/"+str(year)+"/"+str(month)
if os.path.isdir(html_dir):
    file_list = os.listdir(html_dir)
    for file_name in file_list:
        with codecs.open(html_dir+"/"+file_name, "r",encoding="utf-8-sig") as f:
            html = f.read()
            list = file_name.split(".")
            race_id = list[-2]
            race_list, horse_list_list = get_rade_and_horse_data_by_html(race_id, html) # 長くなるので省略
            for horse_list in horse_list_list:
                horse_se = pd.Series( horse_list, index=horse_df.columns)
                horse_df = horse_df.append(horse_se, ignore_index=True)
            race_se = pd.Series(race_list, index=race_df.columns )
            race_df = race_df.append(race_se, ignore_index=True )
            
race_df.to_csv(save_race_csv, header=True, index=False)
horse_df.to_csv(horse_race_csv, header=True, index=False)

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

...

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse_df.append(horse_se, ignore_index=True)
C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\4039284634.py:201: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  horse_df = horse

In [7]:
# 時間情報を抜き出して、日付情報と結合。datetime型にする
race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')
race_df["date"] = race_df["date"] + race_df["time"]
race_df["date"] = pd.to_datetime(race_df['date'], format='%Y年%m月%d日%H時%M分')
# もともとのtimeは不要なので削除
race_df.drop(['time'], axis=1, inplace=True)

# 何ラウンド目かのカラムに余分なRや空白・改行が含まれているので取り除く
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

C:\Users\kawam\AppData\Local\Temp\ipykernel_32808\1109696361.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  race_df["time"] = race_df["time"].str.replace('発走 : (\d\d):(\d\d)(.|\n)*', r'\1時\2分')


In [8]:
race_df.head()

,race_id,race_round,race_title,race_course,weather,ground_status,date,where_racecourse,total_horse_number,frame_number_first,...,hukusyo_second,hukusyo_third,wakuren,umaren,wide_1_2,wide_1_3,wide_2_3,umatan,renhuku3,rentan3
0,201805010101,1,3歳未勝利,ダ左1400m,天候 : 晴,ダート : 不良,2018-01-27 10:10:00,1回東京1日目,16,3,...,250,210,"8,620","8,970","2,500","1,700",810,"22,830","12,060","108,820"
1,201805010102,2,3歳未勝利,ダ左2100m,天候 : 晴,ダート : 不良,2018-01-27 10:40:00,1回東京1日目,16,6,...,220,330,290,510,300,540,"1,440",640,"2,380","5,260"
2,201805010103,3,3歳新馬,ダ左1600m,天候 : 晴,ダート : 不良,2018-01-27 11:10:00,1回東京1日目,16,8,...,130,290,380,390,210,970,810,750,"2,410","7,760"
3,201805010106,6,4歳以上500万下,ダ左1400m,天候 : 晴,ダート : 不良,2018-01-27 13:00:00,1回東京1日目,16,5,...,220,170,760,"1,020",410,390,500,"1,950","1,360","8,060"
4,201805010107,7,3歳500万下,ダ左1600m,天候 : 晴,ダート : 不良,2018-01-27 13:30:00,1回東京1日目,15,7,...,170,250,"1,300","1,260",490,"1,030",670,"2,780","4,690","26,880"
